In [14]:
import numpy as np
from helper import *

In [5]:
from dwave.cloud import Client
api_key = read_api('API_key.txt');
client = Client.from_config(token=api_key)

client.get_solvers()

[StructuredSolver(id='DW_2000Q_6'),
 BQMSolver(id='hybrid_binary_quadratic_model_version2'),
 DQMSolver(id='hybrid_discrete_quadratic_model_version1'),
 StructuredSolver(id='Advantage_system1.1')]

In [11]:
from dwave.system.samplers import DWaveSampler

sampler = DWaveSampler(token=api_key)   

print("Connected to sampler", sampler.solver.name)

Connected to sampler Advantage_system1.1


In [12]:
print("Maximum anneal-schedule points: {}".format(sampler.properties["max_anneal_schedule_points"]))
print("Annealing time range: {}".format(sampler.properties["annealing_time_range"]))

max_slope = 1.0/sampler.properties["annealing_time_range"][0]

print("Maximum slope allowed on this solver is {:.2f}.".format(max_slope))

Maximum anneal-schedule points: 12
Annealing time range: [1, 2000]
Maximum slope allowed on this solver is 1.00.


### Assigned a initial state

In [15]:
h = {v: 0.0 for v in sampler.nodelist}
J = {tuple(c): np.random.choice([-1, 1]) for c in sampler.edgelist}

print("Bias 0 assigned to", len(h), "qubits.")
print("Strengths -1/+1 assigned randomly to",len(J),"couplers.")

Bias 0 assigned to 5436 qubits.
Strengths -1/+1 assigned randomly to 37440 couplers.


In [29]:
from schedule import *

reverse_schedule = make_reverse_anneal_schedule(s_target=0.55, hold_time=100, ramp_up_slope=max_slope)
time_total = reverse_schedule[3][0]

print(reverse_schedule)
print("Total anneal-schedule time is {} us".format(time_total))

[[0.0, 1.0], [2.25, 0.55], [102.25, 0.55], [102.7, 1.0]]
Total anneal-schedule time is 102.7 us


### Perform a forward annealing 

In [34]:
forward_answer = sampler.sample_ising(h, J, 
                                      num_reads=1000, 
                                      annealing_time=time_total, 
                                      label='Test_Notebook',
                                      answer_mode='histogram')
forward_solutions, forward_energies = forward_answer.record.sample, forward_answer.record.energy
i5 = int(1.0/95*len(forward_answer))  

# Zip the answer
initial = dict(zip(forward_answer.variables, forward_answer.record[i5].sample))

print("Lowest energy found: {}".format(forward_answer.record.energy[0]))
print("Average energy is {:.2f} with standard deviation {:.2f}".format(forward_energies.mean(), forward_energies.std()))
print("\nSetting the initial state to a sample with energy: {}".format(forward_answer.record.energy[i5]))

Lowest energy found: -14694.0
Average energy is -14650.21 with standard deviation 15.58

Setting the initial state to a sample with energy: -14682.0


### Perform a backward anneal

In [33]:
reverse_anneal_params = dict(anneal_schedule=reverse_schedule, initial_state=initial, reinitialize_state=False)
reverse_answer = sampler.sample_ising(h, J, 
                                      num_reads=1000, 
                                      label='Test_Notebook',
                                      answer_mode='histogram',
                                     **reverse_anneal_params)
reverse_solutions, reverse_energies = reverse_answer.record.sample, reverse_answer.record.energy

print("Lowest energy found:", reverse_answer.record.energy[0])
print("Average energy is {:.2f} with standard deviation {:.2f}".format(reverse_energies.mean(), reverse_energies.std()))

Lowest energy found: -14690.0
Average energy is -14684.84 with standard deviation 3.28
